# Value Iteration in a grid world

### Import necessary libraries

In [13]:
import numpy as np
import json, operator

### Set default parameters and initialize variables

In [14]:
size_x = 4
size_y = 3
step_cost = -0.04
gamma = 0.5
prob_success = 0.8
convergence = 1
prob_fail = (1 - prob_success) / 2
world_print = [[0 for x in range(size_x)] for y in range(size_y)] 
world_access = [[0 for x in range(size_x)] for y in range(size_y)]  
world_reward = [[0 for x in range(size_x)] for y in range(size_y)]

### Helpfunctions for printing the world

In [3]:
def printRow(arr,y,length):
    print("|",end="")
    for elem in arr[y]:
        print(str(elem).center(length),end="")
        print("|",end="")
    print(end="\n")
    

def printSpace(width, length):
    print("+"+str("").center(length,"-")+"+",end="")
    for w in range(width-2):
        print(str("").center(length,"-")+"+",end="")
    print(str("").center(length,"-")+"+")
    
    
def printWorld(world):
    i=0
    for h in range(len(world)*2+1):
        if(h%2==1):
            printRow(world,i,len(world[0]))
            i+=1
        else:
            printSpace(len(world[0]), len(world[0]))

### Import grid world information from JSON file

In [4]:
def readJson():
    with open('world.json') as json_data:
        d = json.load(json_data)
        print("JSON file content")
        print(d)
        return d
       
        
json_data = readJson() 

JSON file content
{'world': [{'row': [{'cell': ['_', 0, 0]}, {'cell': ['_', 0, 0]}, {'cell': ['_', 0, 0]}, {'cell': ['G', 1, 1]}]}, {'row': [{'cell': ['_', 0, 0]}, {'cell': ['#', -1, 0]}, {'cell': ['_', 0, 0]}, {'cell': ['T', 1, -1]}]}, {'row': [{'cell': ['_', 0, 0]}, {'cell': ['_', 0, 0]}, {'cell': ['_', 0, 0]}, {'cell': ['_', 0, 0]}]}]}


### Distribute grid world information to corresponding variables

In [5]:
def initializeWorld(json_data):
    global world_print
    global world_access
    global world_reward
    i = 0
    for row in json_data['world']:
        j = 0
        for cell in row['row']:
            world_print[i][j]= cell['cell'][0]
            world_access[i][j]= cell['cell'][1]
            world_reward[i][j]= cell['cell'][2]
            j=j+1
        i=i+1
        

initializeWorld(json_data)
print("World")    
printWorld(world_print)
print("Accessibility")
printWorld(world_access)
print("Default rewards")
printWorld(world_reward)

World
+----+----+----+----+
| _  | _  | _  | G  |
+----+----+----+----+
| _  | #  | _  | T  |
+----+----+----+----+
| _  | _  | _  | _  |
+----+----+----+----+
Accessibility
+----+----+----+----+
| 0  | 0  | 0  | 1  |
+----+----+----+----+
| 0  | -1 | 0  | 1  |
+----+----+----+----+
| 0  | 0  | 0  | 0  |
+----+----+----+----+
Default rewards
+----+----+----+----+
| 0  | 0  | 0  | 1  |
+----+----+----+----+
| 0  | 0  | 0  | -1 |
+----+----+----+----+
| 0  | 0  | 0  | 0  |
+----+----+----+----+


### Calculation methods for all possible steps

In [6]:
def up(y_coord, x_coord):
    global world_access
    global world_reward
    
    if(y_coord == 0 or world_access[y_coord - 1][x_coord] < 0):
        return world_reward[y_coord][x_coord]
    return world_reward[y_coord -1][x_coord]
    
def down(y_coord, x_coord):
    global world_access
    global world_reward
    
    if(y_coord == size_y - 1 or world_access[y_coord + 1][x_coord] < 0):
        return world_reward[y_coord][x_coord]
    return world_reward[y_coord + 1][x_coord]


def left(y_coord, x_coord):
    global world_access
    global world_reward
    
    if(x_coord == 0 or world_access[y_coord][x_coord - 1] < 0):
        return world_reward[y_coord][x_coord]
    return world_reward[y_coord][x_coord - 1]


def right(y_coord, x_coord):
    global world_access
    global world_reward
    
    if(x_coord == size_x - 1 or world_access[y_coord ][x_coord + 1] < 0):
        return world_reward[y_coord][x_coord]
    return world_reward[y_coord][x_coord + 1]

### Helpfunction to find the lowest value

In [7]:
def getMinReward():
    global world_reward
    
    rewards = []
    for row in world_reward:
        rewards.append(min(row))
    return min(rewards)

### Calculate one iteration step

In [8]:
def calcStep():
    global world_print
    global world_access
    global world_reward
    
    world_reward_new = []
    world_reward_new = world_reward
    rew={}
    for i in range(len(world_reward_new)):
        for j in range(len(world_reward_new[i])):
            rew["^"] = prob_success * up(i,j) + prob_fail * left(i,j) + prob_fail * right(i,j)
            rew["<"] = prob_success * left(i,j) + prob_fail * up(i,j) + prob_fail * down(i,j)
            rew["v"] = prob_success * down(i,j) + prob_fail * left(i,j) + prob_fail * right(i,j)
            rew[">"] = prob_success * right(i,j) + prob_fail * up(i,j) + prob_fail * down(i,j)
            print_symbol, max_value = max(rew.items(), key=operator.itemgetter(1))
            world_print[i][j] = print_symbol
            world_reward_new[i][j] = step_cost + gamma * max_value
    world_reward = world_reward_new

### Iterate until a suiting policy is found

In [15]:
def valueIteration():
    global world_print
    global world_reward
    global convergence
    
    #while(convergence > getMinReward()):
    while(convergence < 100):
        calcStep()
        printWorld(world_reward)
        print(" \n")
        convergence = convergence + 1
    printWorld(world_print)

In [16]:
valueIteration()

+----+----+----+----+
|-0.04|-0.04|-0.04|-0.04|
+----+----+----+----+
|-0.04|-0.042|-0.042|-0.042|
+----+----+----+----+
|-0.04|-0.042|-0.0421|-0.0421|
+----+----+----+----+
 

+----+----+----+----+
|-0.06|-0.0601|-0.0601|-0.0601|
+----+----+----+----+
|-0.0601|-0.061905|-0.06191000000000001|-0.06191000000000001|
+----+----+----+----+
|-0.0601|-0.06191000000000001|-0.0620405|-0.0620405|
+----+----+----+----+
 

+----+----+----+----+
|-0.070005|-0.07014025|-0.07014050000000001|-0.07014050000000001|
+----+----+----+----+
|-0.07014025|-0.0713665125|-0.07137305000000001|-0.07137305000000001|
+----+----+----+----+
|-0.07014050000000001|-0.07137305000000001|-0.07148687749999999|-0.07148687749999999|
+----+----+----+----+
 

+----+----+----+----+
|-0.07500926249999999|-0.07513153812500001|-0.0751318775|-0.0751318775|
+----+----+----+----+
|-0.07513153812500001|-0.07587444940625|-0.07588015775000001|-0.07588015775000001|
+----+----+----+----+
|-0.0751318775|-0.07588015775000001|-0.075963102762

|-0.07999999999999999|-0.07999999999999999|-0.07999999999999999|-0.07999999999999999|
+----+----+----+----+
|-0.07999999999999999|-0.07999999999999999|-0.07999999999999999|-0.07999999999999999|
+----+----+----+----+
|-0.07999999999999999|-0.07999999999999999|-0.07999999999999999|-0.07999999999999999|
+----+----+----+----+
 

+----+----+----+----+
|-0.07999999999999999|-0.07999999999999999|-0.07999999999999999|-0.07999999999999999|
+----+----+----+----+
|-0.07999999999999999|-0.07999999999999999|-0.07999999999999999|-0.07999999999999999|
+----+----+----+----+
|-0.07999999999999999|-0.07999999999999999|-0.07999999999999999|-0.07999999999999999|
+----+----+----+----+
 

+----+----+----+----+
|-0.07999999999999999|-0.07999999999999999|-0.07999999999999999|-0.07999999999999999|
+----+----+----+----+
|-0.07999999999999999|-0.07999999999999999|-0.07999999999999999|-0.07999999999999999|
+----+----+----+----+
|-0.07999999999999999|-0.07999999999999999|-0.07999999999999999|-0.07999999999999999|
